In [194]:
import cv2
import numpy as np
from itertools import count
from skimage.measure import ransac
from skimage.transform import FundamentalMatrixTransform, EssentialMatrixTransform
import matplotlib.pyplot as plt

vid_path = '5.hevc'
w = 1164
h = 874
F = 910

K = np.array([
    [F, 0, w//2],
    [0, F, h//2],
    [0, 0, 1]])
K_inv = np.linalg.inv(K)
W = np.mat([[0,-1,0],[1,0,0],[0,0,1]],dtype=float)

def normalize(x):
  x = add_third(x)
  #np.dot(Kinv, add_ones(pts).T).T[:, 0:2]
  return (K_inv @ x.T).T[:, 0:2]
def denormalize(x):
  return (K @ x)[:-1]

def add_third(x):
  return np.pad(x, pad_width = ((0,0), (0,1)) ,constant_values=1)
def remove_third(x):
  return x[:,[0,1]]

def generate_frames(vid_path):
    video = cv2.VideoCapture(vid_path, cv2.CAP_FFMPEG)
    _, prev_frame = video.read()
    for t in count():
      ret, curr_frame = video.read()
      if not ret:
        break
      yield prev_frame, curr_frame
      prev_frame = curr_frame
    video.release()
    cv2.destroyAllWindows()

def extractFeatures(frame):
  orb = cv2.ORB_create()
  # only works on b/w images
  pts = cv2.goodFeaturesToTrack(np.mean(frame,axis=2).astype(np.uint8), 3000, 0.01, minDistance=7)
  # we need kp class to feed it into ORB.compute to get descriptors
  kps = [cv2.KeyPoint(x=f[0][0], y=f[0][1], size=10) for f in pts]
  kps, des = orb.compute(frame, kps)
  return np.array([(kp.pt[0], kp.pt[1]) for kp in kps]), des

def bfmatcher(kps, dess):
  bf = cv2.BFMatcher(cv2.NORM_HAMMING)
  # find closest descriptors between frames, hamming norm for ORB (BRIEF, BRISK...)
  # they are binary string types
  # L1, L2 for SIFT/SURF
  matches = bf.knnMatch(dess[0], dess[1], k=2)
  res = []
  # DMatch obj: distance (the lower the better)
  # trainIdx: index of the descriptor in train desc
  # queryIdx: index of the descriptor in query desc
  # imgIdx: index of the train image
  for m,n in matches:
    # Lowe's ratio test
    # https://stackoverflow.com/questions/51197091/how-does-the-lowes-ratio-test-work
    if m.distance < 0.75 * n.distance:
        kp1 = kps[0][m.queryIdx]
        kp2 = kps[1][m.trainIdx]
        res.append((kp1, kp2))
  res = np.array(res)
  # prune the outliers by fitting
  assert len(res)>=8, 'not enough points'
  model, inliers = ransac((normalize(res[:,0]), normalize(res[:,1])),
                          #FundamentalMatrixTransform, 
                          EssentialMatrixTransform,
                          min_samples=8,
                          residual_threshold=0.0005, 
                          max_trials=500)

  U, D, Vt = np.linalg.svd(model.params)

  if np.linalg.det(U) < 0:
    U *= -1.0
  if np.linalg.det(Vt) < 0:
    Vt *= -1.0
  R1 = U @ W @ Vt
  R2 = U @ W.T @ Vt
  if abs(np.sum(R1.diagonal())-3) > abs(np.sum(R2.diagonal())-3):
    R = R2
  else: 
    R = R1
  t = U[:, 2]
  if t[2] < 0:
    t *= -1
  print (t)
  #print (np.linalg.det(Vt))
  #print (f'good kps: {len(res[inliers])/len(res)*100:.2f}% out of {len(res)}')
  return res[inliers,0], res[inliers,1]

In [195]:
for i, (p, c) in enumerate(generate_frames(vid_path)):
  
  kps_1, des_1 = extractFeatures(p)
  kps_2, des_2 = extractFeatures(c)

  p1, p2 = bfmatcher([kps_1, kps_2], [des_1, des_2])
  for p in p2:
    cv2.circle(c, (int(p[0]), int(p[1])), 1, (255,255,0)) 
  
  for k1, k2 in zip(p1, p2):
    cv2.line(c, tuple(k1.astype(int)), tuple(k2.astype(int)), (0,200,200), 1)
  
  cv2.imshow('v', c)
  key = cv2.waitKey(1)
  if key == ord('q'):
    break

cv2.destroyAllWindows()

[ 0.02864063 -0.02461174  0.99928673]
[0.06006411 0.00157349 0.99819328]
[ 0.15616058 -0.00530769  0.98771742]
[ 0.11393956 -0.06094295  0.99161673]
[ 0.05745531 -0.00720755  0.99832206]
[ 0.09025534 -0.01131766  0.99585435]
[ 0.06057845 -0.028605    0.99775348]
[0.07685426 0.00337596 0.99703662]
[ 0.04531159 -0.02111179  0.99874979]
[6.55560915e-02 8.77396226e-04 9.97848500e-01]
[ 0.33999673 -0.09674071  0.93543757]
[-0.14301247  0.02903353  0.98929494]
[-0.54830206  0.05090046  0.83472989]
[ 0.04237741 -0.0091096   0.99906014]
[ 0.07923492 -0.0249542   0.99654358]
[-0.01568548 -0.0170821   0.99973105]
[-0.03200409 -0.0146281   0.99938069]
[ 0.19121555 -0.05573728  0.97996427]
[ 0.27747831 -0.00182609  0.96073017]
[ 0.02624022 -0.01612213  0.99952565]
[ 0.04868183 -0.01114623  0.99875214]
[ 0.03713938 -0.00193583  0.99930822]
[0.03955781 0.00335504 0.99921165]
[ 0.32570123 -0.0084646   0.94543485]
[ 0.0039211  -0.01561374  0.99987041]
[ 0.05217223 -0.0221368   0.99839272]
[ 0.12105149